In [1]:
import pandas as pd
from tqdm.auto import tqdm
import editdistance

In [2]:
df_train = pd.read_csv("../dataset/original/entity-resolution_advanced-topics-training_data.csv")
df_train.head(10)

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4.102971e+12,imquires@zoho.vo,typo,10111147
1,12172071-T0,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,NaN,NaN,typo,12172071
2,10104491-T1,MOONTA HOLDINGS LTD.,entity,NaN,6.501346e+10,NaN,typo,10104491
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,NaN,consulting@zoho.de,original,10068808
4,13011715,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,NaN,NaN,original,13011715
5,10105687,ODIN GROUP INVESTMENTS S.A.,entity,NaN,4.966891e+10,info@outlook.cz,original,10105687
6,10016805-T2,CAPRI SOLUTIONS LIMITED,entity,NaN,NaN,help@CAPRI.SOLUTIONS.LxMITED.de,typo,10016805
7,12133421,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",NaN,CrossmatchSA@aol.gov,original,12133421
8,12126772,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,NaN,support@LAURENCE.POUNTNEY.LTD.gov,original,12126772
9,11002523,"BREIT, MARCUS",intermediary,NaN,4.401803e+11,NaN,original,11002523


In [3]:
df_name = df_train[['record_id','name','linked_id']]
df_name.name = df_name.name.astype(str)
df_name.name = df_name.name.str.lower()
df_name.head(10)

c:\users\blueace17\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,record_id,name,linked_id
0,10111147-T1,ardia enterprisesmltd.,10111147
1,12172071-T0,andré almeida blanco,12172071
2,10104491-T1,moonta holdings ltd.,10104491
3,10068808,springfield investments group limited,10068808
4,13011715,grigory uvarov,13011715
5,10105687,odin group investments s.a.,10105687
6,10016805-T2,capri solutions limited,10016805
7,12133421,crossmatch s.a.,12133421
8,12126772,laurence pountney ltd,12126772
9,11002523,"breit, marcus",11002523


In [4]:
blacklist = [' ltd ', ' ltd.', ' ltd', 'enterprises', 'enterprise', ' limited', ' limited ', ' inc ', ' inc. ', ' inc.', 
            ' s.a. ', ' s.a.', 'international', 'holdings', 'investments', ' corp.', ' corp. ', 'corp. ', ' investment ', 
             ' investment', 'investment ', ' management ', 'management ', ' management', 'development', ' group ', ' group',
             'group ', ' trading', ' trading ', 'trading ', ' services', 'services ', ' services', ' holding', ' holding ', 
             'holding ', "  "]
for w in tqdm(blacklist):
    df_name.name = df_name.name.str.replace(w, ' ')
df_name.head(10)

,record_id,name,linked_id
0,10111147-T1,ardia mltd.,10111147
1,12172071-T0,andré almeida blanco,12172071
2,10104491-T1,moonta,10104491
3,10068808,springfield,10068808
4,13011715,grigory uvarov,13011715
5,10105687,odin,10105687
6,10016805-T2,capri solutions,10016805
7,12133421,crossmatch,12133421
8,12126772,laurence pountney,12126772
9,11002523,"breit, marcus",11002523


In [5]:
dict_dupl = {}

In [6]:
for (i,n) in tqdm(zip(df_name.record_id, df_name.name)):
    if n in dict_dupl.keys():
        dict_dupl[n].append(i)
    else:
        dict_dupl[n] = [i]

In [7]:
len(list(dict_dupl))

314869

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_name.iloc[:,:-1], 
                                                    df_name.linked_id, test_size=0.3, random_state=17)

In [9]:
X_train = X_train.sort_index()
X_train.head(10)

,record_id,name
0,10111147-T1,ardia mltd.
1,12172071-T0,andré almeida blanco
2,10104491-T1,moonta
4,13011715,grigory uvarov
5,10105687,odin
6,10016805-T2,capri solutions
7,12133421,crossmatch
8,12126772,laurence pountney
9,11002523,"breit, marcus"
10,10046375-NV0,afccotrade


In [10]:
y_train = y_train.sort_index()
y_train.head(10)

0     10111147
1     12172071
2     10104491
4     13011715
5     10105687
6     10016805
7     12133421
8     12126772
9     11002523
10    10046375
Name: linked_id, dtype: int64

In [11]:
X_test = X_test.sort_index()
X_test.head(10)

,record_id,name
3,10068808,springfield
14,12188364,li guobang
15,10151690,zucean
16,10117892-T4,longxi
19,10045048-M0,"sandro botticelli,"
20,12220469-M2,"ali ahmed darrat, najib ahmed darrat, ahmed aa..."
25,12214184,santiago peña durini
26,13012085,lio wen tong
29,10022235-M2,spirit ride
30,10198754-M1,inversiones siglo veintiuno


In [12]:
y_test = y_test.sort_index()
y_test.head(10)

3     10068808
14    12188364
15    10151690
16    10117892
19    10045048
20    12220469
25    12214184
26    13012085
29    10022235
30    10198754
Name: linked_id, dtype: int64

In [65]:
for(i, n) in tqdm(zip(X_test.index, X_test.name)):
    #print(f'{i} {n}')
    for (rid, row) in zip(X_train.record_id, X_train.name):
        if (len(n) - editdistance.eval(n, row))/len(n) > 0.85:
            label = rid.split('-')[0]
            #print(f'{rid} - {row}: label {label}')

KeyboardInterrupt: 

In [16]:
import string
import unidecode
def create_name_letters_matrix(df):
    df = df[['record_id','name']]
    df.name = df.name.astype(str) # convert to string
    df.name = df.name.str.lower() # lowercase
    df.name = df.name.str.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    # remove accented letters
    no_accents = []
    for s in df.name:
        no_accents.append(unidecode.unidecode(s))
    df.name = no_accents
    # create return matrix
    columns = ['record_id','name','a','b','c','d','e','f','g','h','i','j','k','l',
               'm','n','o','p','q','r','s','t','u','v','w','x','y','z']
    name_letters_matrix = pd.DataFrame(columns=columns)
    name_letters_matrix.record_id = df.record_id.copy()
    name_letters_matrix.name = df.name.copy()
    # count occurence of each letter and add the columns to the return df
    for l in tqdm(['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                   'o','p','q','r','s','t','u','v','w','x','y','z']):
        new_col = []
        for (i,n) in zip(name_letters_matrix.index, name_letters_matrix.name):
            new_col.append(n.count(l))
        name_letters_matrix[l] = new_col
    return name_letters_matrix

In [17]:
nlm_train = create_name_letters_matrix(X_train)
nlm_test = create_name_letters_matrix(X_test)

In [53]:
nlm_train.head(10)

,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
0,10111147-T1,ardia mltd,2,0,0,2,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1,12172071-T0,andre almeida blanco,4,1,1,2,2,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,10104491-T1,moonta,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,13011715,grigory uvarov,1,0,0,0,0,0,2,0,...,0,3,0,0,1,2,0,0,1,0
5,10105687,odin,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,10016805-T2,capri solutions,1,0,1,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
7,12133421,crossmatch,1,0,2,0,0,0,0,1,...,0,1,2,1,0,0,0,0,0,0
8,12126772,laurence pountney,1,0,1,0,3,0,0,0,...,0,1,0,1,2,0,0,0,1,0
9,11002523,breit marcus,1,1,1,0,1,0,0,0,...,0,2,1,1,1,0,0,0,0,0
10,10046375-NV0,afccotrade,2,0,2,1,1,1,0,0,...,0,1,0,1,0,0,0,0,0,0


In [52]:
nlm_train[nlm_train.index == 139758]

,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
139758,10031869-NV0,springfield,0,0,0,1,1,1,1,0,...,0,1,1,0,0,0,0,0,0,0


In [21]:
nlm_test.head(5)

,record_id,name,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
3,10068808,springfield,0,0,0,1,1,1,1,0,...,0,1,1,0,0,0,0,0,0,0
14,12188364,li guobang,1,1,0,0,0,0,2,0,...,0,0,0,0,1,0,0,0,0,0
15,10151690,zucean,1,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
16,10117892-T4,longxi,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
19,10045048-M0,sandro botticelli,1,1,1,1,1,0,0,0,...,0,1,1,2,0,0,0,0,0,0


In [20]:
from scipy.sparse import *
from scipy import *
matrix = csr_matrix(nlm_train.drop(['record_id','name'], axis=1))

<484008x26 sparse matrix of type '<class 'numpy.int64'>'
	with 4429556 stored elements in Compressed Sparse Row format>

In [31]:
nlm_test_letters = nlm_test.drop(['record_id','name'], axis=1)
record = csr_matrix(nlm_test_letters.iloc[0,:])

In [66]:
csr_mmm = csr_matrix( nlm_test.drop(['record_id','name'], axis=1))

In [ ]:
final = sim.cosine(csr_mmm, matrix.T, k=10)


Computing:   1%|█▍                                                                                                                                                                            | 1656/207432 [00:08<18:15, 187.87it/s]

In [51]:
final.argmax()

139758

In [56]:
final.max()

0.94174194

In [62]:
import numpy as np
arr = np.array(final)
arr

array(<1x691440 sparse matrix of type '<class 'numpy.float32'>'
	with 10 stored elements in COOrdinate format>, dtype=object)